In [61]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [5]:
data_path = r'C:\Users\taanh\Downloads\cmsa_combined.csv'
info_path = r'C:\Users\taanh\Downloads\crowd-prediction-laura\data\sensors-info.csv'
forecast_path = r'C:\Users\taanh\Downloads\crowd-prediction-laura\experiments\fake_forecasts_for_frontend\GAWW-11.csv'

In [4]:
forecast_path = r'C:\Users\nonta\Dropbox\My PC (LAPTOP-VMKUGGQ6)\Desktop\crowd-prediction-laura\experiments\fake_forecasts_for_frontend\GAWW-11.csv'
info_path = r'C:\Users\nonta\Dropbox\My PC (LAPTOP-VMKUGGQ6)\Desktop\crowd-prediction-laura\data\sensors-info.csv'

In [6]:
#data=pd.read_csv(data_path)
info=pd.read_csv(info_path)
forecast=pd.read_csv(forecast_path)

In [7]:
display(info.head())
display(forecast.head())

,Unnamed: 0,objectnummer,location_name,gebied,crowd_threshold_low,crowd_threshold_high
0,23,CMSA-GAWW-11,Korte Niezel,Red_light,1071.0,1377.0
1,24,CMSA-GAWW-12,Oudekennissteeg,Red_light,819.0,1053.0
2,26,CMSA-GAWW-14,Oudezijds Voorburgwal t.h.v. 91,Red_light,1134.0,1458.0


,datetime,prediction,upper,lower
0,2021-12-01 00:00:00,21.789772,26.147726,17.431818
1,2021-12-01 00:15:00,6.112484,7.334981,4.889987
2,2021-12-01 00:30:00,15.331353,18.397623,12.265082
3,2021-12-01 00:45:00,10.883493,13.060191,8.706794
4,2021-12-01 01:00:00,9.907216,11.888659,7.925773


In [27]:
date_time_str = forecast['datetime'].iloc[0]
dd = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
dd.date()

datetime.date(2021, 12, 1)

In [24]:
date_time_str = forecast['datetime'].iloc[-1]
datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2021, 12, 6, 23, 45)

In [22]:
sensors_info = info
loc_forecast = forecast
crowds={}

# Analyzed sensors:
locations = {'GAWW-11': [52.374611, 4.899833],
             'GAWW-12': [52.373883, 4.898653],
             'GAWW-14': [52.373571, 4.898272]}

loc = 'GAWW-12'
ind=1

# for ind, loc in enumerate(locations):
loc_info = sensors_info[sensors_info['objectnummer']=='CMSA-'+loc]

# Get the times with yellow forecast:
yellow_forecast = loc_forecast[loc_forecast['prediction'] > loc_info['crowd_threshold_low'][ind]-500]
if yellow_forecast.shape[0] > 0:
    yellow_forecast = yellow_forecast[yellow_forecast['prediction'] < loc_info['crowd_threshold_high'][ind]-600]

# Get the times with red forecast:
red_forecast = loc_forecast[loc_forecast['prediction'] > loc_info['crowd_threshold_high'][ind]-600]

crowds[loc] = {'yellow': yellow_forecast, 'red':red_forecast}
crowds

{'GAWW-12': {'yellow':                 datetime  prediction       upper       lower
  259  2021-12-03 16:45:00  353.026852  423.632222  282.421481
  260  2021-12-03 17:00:00  320.541385  384.649662  256.433108
  343  2021-12-04 13:45:00  342.594453  411.113343  274.075562
  344  2021-12-04 14:00:00  356.849769  428.219723  285.479815
  345  2021-12-04 14:15:00  358.164405  429.797286  286.531524
  346  2021-12-04 14:30:00  383.009197  459.611036  306.407358
  347  2021-12-04 14:45:00  339.703492  407.644190  271.762793
  351  2021-12-04 15:45:00  322.733505  387.280205  258.186804
  353  2021-12-04 16:15:00  388.222668  465.867201  310.578134
  354  2021-12-04 16:30:00  324.352592  389.223110  259.482074
  358  2021-12-04 17:30:00  414.653314  497.583977  331.722651
  438  2021-12-05 13:30:00  320.508870  384.610643  256.407096
  440  2021-12-05 14:00:00  343.024132  411.628958  274.419306
  441  2021-12-05 14:15:00  336.959287  404.351145  269.567430
  443  2021-12-05 14:45:00  332.98

In [52]:
yellow = yellow_forecast.drop(['upper', 'lower'], axis=1)
yellow['overcrowdedness'] = 'yellow'
yellow.set_index('datetime')

red = red_forecast.drop(['upper', 'lower'], axis=1)
red['overcrowdedness'] = 'red'
red.set_index('datetime')

merged = pd.concat([yellow, red])
merged = merged.sort_values(by=['datetime'])

In [53]:
merged['start'] = pd.to_datetime(merged['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
merged['end'] = pd.to_datetime(merged['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')

In [88]:
merged

,datetime,prediction,overcrowdedness,start,end
259,2021-12-03 16:45:00,353.026852,yellow,2021-12-03 16:45:00,2021-12-03 16:45:00
260,2021-12-03 17:00:00,320.541385,yellow,2021-12-03 17:00:00,2021-12-03 17:00:00
343,2021-12-04 13:45:00,342.594453,yellow,2021-12-04 13:45:00,2021-12-04 13:45:00
344,2021-12-04 14:00:00,356.849769,yellow,2021-12-04 14:00:00,2021-12-04 14:00:00
345,2021-12-04 14:15:00,358.164405,yellow,2021-12-04 14:15:00,2021-12-04 14:15:00
346,2021-12-04 14:30:00,383.009197,yellow,2021-12-04 14:30:00,2021-12-04 14:30:00
347,2021-12-04 14:45:00,339.703492,yellow,2021-12-04 14:45:00,2021-12-04 14:45:00
351,2021-12-04 15:45:00,322.733505,yellow,2021-12-04 15:45:00,2021-12-04 15:45:00
353,2021-12-04 16:15:00,388.222668,yellow,2021-12-04 16:15:00,2021-12-04 16:15:00
354,2021-12-04 16:30:00,324.352592,yellow,2021-12-04 16:30:00,2021-12-04 16:30:00


In [100]:
rowlist = []
last_row = None
last_time = None
cut_row = None

for row in merged.itertuples():
    this_row = row._asdict()
    if cut_row is None:
        cut_row = row._asdict()
    elif not(cut_row.get('overcrowdedness') == row.overcrowdedness):
        rowlist.append(cut_row)
        cut_row = row._asdict()
    elif (row.start - cut_row.get('end'))< timedelta(minutes=16):
        cut_row['end'] = row.end
    else:
        rowlist.append(cut_row)
        cut_row = row._asdict()

    
rowlist.append(cut_row)
combine_merged = pd.DataFrame.from_dict(rowlist)
combine_merged

,Index,datetime,prediction,overcrowdedness,start,end
0,259,2021-12-03 16:45:00,353.026852,yellow,2021-12-03 16:45:00,2021-12-03 17:00:00
1,343,2021-12-04 13:45:00,342.594453,yellow,2021-12-04 13:45:00,2021-12-04 14:45:00
2,351,2021-12-04 15:45:00,322.733505,yellow,2021-12-04 15:45:00,2021-12-04 15:45:00
3,353,2021-12-04 16:15:00,388.222668,yellow,2021-12-04 16:15:00,2021-12-04 16:30:00
4,355,2021-12-04 16:45:00,456.781209,red,2021-12-04 16:45:00,2021-12-04 17:15:00
5,358,2021-12-04 17:30:00,414.653314,yellow,2021-12-04 17:30:00,2021-12-04 17:30:00
6,438,2021-12-05 13:30:00,320.508870,yellow,2021-12-05 13:30:00,2021-12-05 13:30:00
7,440,2021-12-05 14:00:00,343.024132,yellow,2021-12-05 14:00:00,2021-12-05 14:15:00
8,443,2021-12-05 14:45:00,332.988723,yellow,2021-12-05 14:45:00,2021-12-05 14:45:00
9,449,2021-12-05 16:15:00,341.886104,yellow,2021-12-05 16:15:00,2021-12-05 16:45:00


In [ ]:
        print(this_row['start'] - last_row['end'])
        if (this_row['start'] - last_row['end']) < timedelta(minutes=16):
            print('Concat')
            print(this_row['start'] - last_row['end'])
            print(last_row['end']) 
            print('----')
    last_row = this_row

In [125]:
combined = combine_merged.drop(['Index', 'datetime'], axis=1)
combined.start.dt.date
combined['start_time'] = combined.start.dt.time
combined['end_time'] = combined.end.dt.time
combined['date'] = combined.start.dt.date
combined.date.unique()

array([datetime.date(2021, 12, 3), datetime.date(2021, 12, 4),
       datetime.date(2021, 12, 5), datetime.date(2021, 12, 6)],
      dtype=object)

In [130]:
combined.start.dt.floor('Min')

0    2021-12-03 16:45:00
1    2021-12-04 13:45:00
2    2021-12-04 15:45:00
3    2021-12-04 16:15:00
4    2021-12-04 16:45:00
5    2021-12-04 17:30:00
6    2021-12-05 13:30:00
7    2021-12-05 14:00:00
8    2021-12-05 14:45:00
9    2021-12-05 16:15:00
10   2021-12-05 17:00:00
11   2021-12-06 16:45:00
Name: start, dtype: datetime64[ns]

In [134]:
len(combined)

12

In [70]:
time = datetime.strptime('2021-12-04 14:30:00', '%Y-%m-%d %H:%M:%S')
str(time) in crowds['GAWW-12']['yellow'].datetime.tolist()

False

In [66]:
list = crowds['GAWW-12']['yellow'].datetime.tolist()
list

['2021-12-03 16:45:00',
 '2021-12-03 17:00:00',
 '2021-12-04 13:45:00',
 '2021-12-04 14:00:00',
 '2021-12-04 14:15:00',
 '2021-12-04 14:30:00',
 '2021-12-04 14:45:00',
 '2021-12-04 15:45:00',
 '2021-12-04 16:15:00',
 '2021-12-04 16:30:00',
 '2021-12-04 17:30:00',
 '2021-12-05 13:30:00',
 '2021-12-05 14:00:00',
 '2021-12-05 14:15:00',
 '2021-12-05 14:45:00',
 '2021-12-05 16:15:00',
 '2021-12-05 16:30:00',
 '2021-12-05 16:45:00',
 '2021-12-06 16:45:00',
 '2021-12-06 17:00:00']

In [22]:
data[data['GAWW-11']>1071.0].drop(['GAWW-12', 'GAWW-14'], axis=1)

,Unnamed: 0,datetime,GAWW-11
36758,36758,18/09/2021 21:30,1159
36760,36760,18/09/2021 22:00,1135
40791,40791,30/10/2021 21:45,1173
40793,40793,30/10/2021 22:15,1250
40794,40794,30/10/2021 22:30,1246
40795,40795,30/10/2021 22:45,1236
40796,40796,30/10/2021 23:00,1133
40797,40797,30/10/2021 23:15,1104
40798,40798,30/10/2021 23:30,1125
40799,40799,30/10/2021 23:45,1074


In [27]:
data[data['GAWW-12']>1053.0].drop(['GAWW-11', 'GAWW-14'], axis=1).count()

Unnamed: 0    27
datetime      27
GAWW-12       27
dtype: int64

In [24]:
data[data['GAWW-14']>1134.0].drop(['GAWW-11', 'GAWW-12'], axis=1)

,Unnamed: 0,datetime,GAWW-14
40792,40792,30/10/2021 22:00,1833


In [81]:
forecast1 = r'C:\Users\nonta\Dropbox\My PC (LAPTOP-VMKUGGQ6)\Desktop\crowd-prediction-laura\experiments\fake_forecasts_for_frontend\GAWW-11.csv'
forecast2 = r'C:\Users\nonta\Dropbox\My PC (LAPTOP-VMKUGGQ6)\Desktop\crowd-prediction-laura\experiments\fake_forecasts_for_frontend\GAWW-12.csv'
forecast3 = r'C:\Users\nonta\Dropbox\My PC (LAPTOP-VMKUGGQ6)\Desktop\crowd-prediction-laura\experiments\fake_forecasts_for_frontend\GAWW-14.csv'
forecast1=pd.read_csv(forecast1).set_index('datetime')
forecast2=pd.read_csv(forecast2).set_index('datetime')
forecast3=pd.read_csv(forecast3).set_index('datetime')

In [90]:
all_forecasts_dict = forecast1.drop(['upper', 'lower'], axis=1)
all_forecasts_dict = all_forecasts_dict.rename(columns={'prediction': 'GAWW-11'})
all_forecasts_dict['GAWW-12'] = forecast2.prediction
all_forecasts_dict['GAWW-14'] = forecast3.prediction
all_forecasts_dict

,GAWW-11,GAWW-12,GAWW-14
datetime,,,
2021-12-01 00:00:00,21.789772,17.346223,11.348152
2021-12-01 00:15:00,6.112484,14.876736,11.875427
2021-12-01 00:30:00,15.331353,7.307850,15.218409
2021-12-01 00:45:00,10.883493,8.513091,6.745491
2021-12-01 01:00:00,9.907216,18.606734,3.124113
...,...,...,...
2021-12-06 22:45:00,26.301531,21.448530,24.820412
2021-12-06 23:00:00,28.560800,8.398890,20.166938
2021-12-06 23:15:00,25.688460,6.854641,20.300598


In [89]:
all_forecasts_dict.index = pd.to_datetime(all_forecasts_dict.index)
all_forecasts_dict

,GAWW-11,GAWW-12,GAWW-14
datetime,,,
2021-12-01 00:00:00,21.789772,17.346223,11.348152
2021-12-01 00:15:00,6.112484,14.876736,11.875427
2021-12-01 00:30:00,15.331353,7.307850,15.218409
2021-12-01 00:45:00,10.883493,8.513091,6.745491
2021-12-01 01:00:00,9.907216,18.606734,3.124113
...,...,...,...
2021-12-06 22:45:00,26.301531,21.448530,24.820412
2021-12-06 23:00:00,28.560800,8.398890,20.166938
2021-12-06 23:15:00,25.688460,6.854641,20.300598


In [100]:
all_forecasts_dict['datetime2']=pd.to_datetime(all_forecasts_dict.index)
all_forecasts_dict

,GAWW-11,GAWW-12,GAWW-14,datetime,datetime2
datetime,,,,,
2021-12-01 00:00:00,21.789772,17.346223,11.348152,2021-12-01 00:00:00,2021-12-01 00:00:00
2021-12-01 00:15:00,6.112484,14.876736,11.875427,2021-12-01 00:15:00,2021-12-01 00:15:00
2021-12-01 00:30:00,15.331353,7.307850,15.218409,2021-12-01 00:30:00,2021-12-01 00:30:00
2021-12-01 00:45:00,10.883493,8.513091,6.745491,2021-12-01 00:45:00,2021-12-01 00:45:00
2021-12-01 01:00:00,9.907216,18.606734,3.124113,2021-12-01 01:00:00,2021-12-01 01:00:00
...,...,...,...,...,...
2021-12-06 22:45:00,26.301531,21.448530,24.820412,2021-12-06 22:45:00,2021-12-06 22:45:00
2021-12-06 23:00:00,28.560800,8.398890,20.166938,2021-12-06 23:00:00,2021-12-06 23:00:00
2021-12-06 23:15:00,25.688460,6.854641,20.300598,2021-12-06 23:15:00,2021-12-06 23:15:00


In [99]:
all_forecasts_dict.index

Index(['2021-12-01 00:00:00', '2021-12-01 00:15:00', '2021-12-01 00:30:00',
       '2021-12-01 00:45:00', '2021-12-01 01:00:00', '2021-12-01 01:15:00',
       '2021-12-01 01:30:00', '2021-12-01 01:45:00', '2021-12-01 02:00:00',
       '2021-12-01 02:15:00',
       ...
       '2021-12-06 21:30:00', '2021-12-06 21:45:00', '2021-12-06 22:00:00',
       '2021-12-06 22:15:00', '2021-12-06 22:30:00', '2021-12-06 22:45:00',
       '2021-12-06 23:00:00', '2021-12-06 23:15:00', '2021-12-06 23:30:00',
       '2021-12-06 23:45:00'],
      dtype='object', name='datetime', length=576)